In [28]:
from azureml.core import Workspace, Experiment

ws = Workspace.get(name="quick-starts-ws-121649")
exp = Experiment(workspace=ws, name="quick-starts-ws-121649")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

# run = exp.start_logging()

Workspace name: quick-starts-ws-121649
Azure region: southcentralus
Subscription id: bc4ac49b-f3f0-45bb-995b-638f2c80696f
Resource group: aml-quickstarts-121649


In [29]:
from azureml.core.compute import ComputeTarget, AmlCompute

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

comptarg = ComputeTarget(workspace=ws, name='Ranga')
comp_config = AmlCompute.provisioning_configuration(vm_size="Standard_D2_V2",max_nodes=4)
comp_cluster = ComputeTarget.create(ws,'cluster',comp_config)

In [56]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy, MedianStoppingPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform, randint, choice
from sklearn.metrics import mean_squared_error,accuracy_score
import os

# Specify parameter sampler
ps = RandomParameterSampling( {
    'C': uniform(0.05,0.2),
    'max_iter': choice(50,100,150,200)
})

# Specify a Policy
policy = BanditPolicy(slack_factor=0.01)

if "training" not in os.listdir():
    os.mkdir("./training")

# Create a SKLearn estimator for use with train.py
est = SKLearn(entry_script='train_LATEST (1).py',source_directory='./training',compute_target=comp_cluster)

# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(hyperparameter_sampling=ps,primary_metric_name='Accuracy',primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,max_total_runs=20,policy=policy,estimator=est)

In [58]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

exp = Experiment(workspace=ws,name='Ranga_Hyperdrive')
run = exp.submit(hyperdrive_config,show_output=True)
RunDetails(run).show()

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

In [59]:
print(run.get_portal_url())

https://ml.azure.com/experiments/Ranga_Hyperdrive/runs/HD_ca3fbed6-2606-454d-8fa0-e3f89dab41a5?wsid=/subscriptions/bc4ac49b-f3f0-45bb-995b-638f2c80696f/resourcegroups/aml-quickstarts-121649/workspaces/quick-starts-ws-121649


In [60]:
import joblib
# Get your best run and save the model from that run.

best = run.get_best_run_by_primary_metric()
joblib.dump(best,'best.py')

TypeError: can't pickle _thread.RLock objects

In [13]:
from azureml.data.dataset_factory import TabularDatasetFactory
import pandas as pd
# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

ds = pd.read_csv('https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv')

In [24]:
from train_LATEST import clean_data
from sklearn.model_selection import train_test_split

# Use the clean_data function to clean your data.
x, y = clean_data(ds)
cleands = x.join(y)

cleandstrain,cleandstest = train_test_split(cleands,test_size=0.33,random_state=42)

In [26]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task='regression',
    primary_metric='normalized_root_mean_squared_error',
    training_data=cleandstrain,
    label_column_name='y',
    n_cross_validations=3)

In [29]:
# Submit your automl run

autoexp = Experiment(workspace=ws,name='Ranga_AutoML')
autorun = autoexp.submit(automl_config)
RunDetails(autorun).show()

WARNING - Running on local machine. Note that local runs always run synchronously even if you use the parameter 'show_output=False'


[01:53:03] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[01:53:03] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[01:53:03] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[01:53:13] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[02:14:43] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[02:14:46] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

In [30]:
# Retrieve and save your best automl model.

best = autorun.get_best_run_by_primary_metric()
joblib.dump(best,'best.py')

AttributeError: 'AutoMLRun' object has no attribute 'get_best_run_by_primary_metric'